# Modelos de Machine Learning: Benchmarks

## Intro

### Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import math
import altair as alt
import joblib

### Carga de Datos

In [3]:
def load_data(df_in):
    df = pd.read_csv(df_in+'.csv')
    df = df.drop("Unnamed: 0", axis=1)
    return df

# Cargar los datos
df = load_data('df_C_S_v3')
df = df[df['Client'] != 'Boxto']
#df = df[df['Client'] != 'AOV']
df = df.reset_index()

## CPC

### CPC: Elección Output y features 

In [5]:
Variable_Target = 'CPC'

# Calcular la media y la desviación estándar de la variable objetivo y
mean_y, std_y = np.mean(df[Variable_Target]), np.std(df[Variable_Target])
outlier_threshold = 3 * std_y

max_y = mean_y + (3 * std_y)
min_y = mean_y - (3 * std_y)

df = df[df[Variable_Target] <= max_y]
df = df[df[Variable_Target] >= min_y]

In [6]:
X = df.copy()

# Preparar los datos: Features
X = X[['Año','Mes', 'Objective', 'Cost', 'Country', 'Media_type', 'Traffic_source', 'Client','Format_New','Platform','Strategy','Plataforma','Campaign_Type','Ecommerce','Service_Product']]
       #'Tipo Search', 'Bench Gral CPC',       #'Bench Search CPC', 'Bench GralSch CPL', 'Bench Search CPL',       #'Bench GralSch CTR', 'Bench Search CTR', 'Bench GralSch CR',       #'Bench Search AvgCR', 'Tipo FB', 'Bench GralFB CPC', 'Bench FB CPC',
       #'Bench GralFB CPAction', 'Bench FB CPAction', 'Bench GralFB CTR',       #'Bench FB CTR', 'Bench GralFB CR', 'Bench FB AvgCR', 'Tipo YT',       #'Bench GralYT CPV', 'Bench YT CPV', 'Bench GralYT CTR', 'Bench YT CTR',       #'Bench GralYT VR', 'Bench FB AvgVR']]

X = X.reset_index()

categorical_features = ['Objective', 'Country', 'Media_type', 'Traffic_source', 'Client','Format_New','Platform','Strategy','Plataforma','Campaign_Type','Ecommerce','Service_Product']  #,'Tipo Search','Tipo FB','Tipo YT']

# Preprocesamiento de variables categóricas
X_dum = pd.get_dummies(X, columns=categorical_features)

#Elijo el valor a predecir. CPC: La hipótesis es que el valor de COST es conocido y controlable: El output en rigor será Clicks y expresaremos el resultado en CPC Cost/Clicks
y = df[Variable_Target]

In [7]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X_dum[['Año','Mes','Cost']])

pca = PCA(n_components=2)  # Aquí estamos conservando solo una componente principal
X_pca = pca.fit_transform(X_Scaled)
X_pca = pd.DataFrame(X_pca)

X_dum['X_pca_0'] = X_pca[0]
X_dum['X_pca_1'] = X_pca[1]
X['X_pca_0'] = X_pca[0]
X['X_pca_1'] = X_pca[1]

In [8]:
joblib.dump(scaler, 'scaler_model.joblib')
joblib.dump(pca, 'pca_model.joblib')

['pca_model.joblib']

In [9]:
# Separo en conjuntos de Train y de Test
X_train = X_dum

### XGBoost CPC

Version 14 ( MSE_test = 0.06 | RMSE_test = 0.25 | MAE_test = 0.14 | r2= 0.78)

In [37]:
import xgboost as xgb

In [38]:
xgboost = xgb.XGBRegressor(learning_rate=0.08, max_depth=8, n_estimators=60)

In [39]:
xgboost.fit(X_train.drop('index',axis=1), y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=60, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [40]:
# save to JSON
xgboost.save_model("model_xgboost_CPC.json")
interval_cpc = 0.27  #CL 85%

### Tuning

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [22]:
# Definir los hiperparámetros a ajustar
parameters = {'learning_rate': [0.06, 0.07, 0.08],
              'max_depth': [6, 8, 10],
              'n_estimators': [30, 40, 50, 60]}

In [23]:
# Configurar la validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [24]:
# Realizar la búsqueda de parámetros
grid_search = GridSearchCV(estimator=xgboost, param_grid=parameters, cv=kfold, scoring='neg_mean_squared_error', verbose = 3)
grid_result = grid_search.fit(X_train, y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END learning_rate=0.06, max_depth=6, n_estimators=30;, score=-0.066 total time=   0.0s
[CV 2/5] END learning_rate=0.06, max_depth=6, n_estimators=30;, score=-0.074 total time=   0.0s
[CV 3/5] END learning_rate=0.06, max_depth=6, n_estimators=30;, score=-0.065 total time=   0.0s
[CV 4/5] END learning_rate=0.06, max_depth=6, n_estimators=30;, score=-0.058 total time=   0.0s
[CV 5/5] END learning_rate=0.06, max_depth=6, n_estimators=30;, score=-0.058 total time=   0.0s
[CV 1/5] END learning_rate=0.06, max_depth=6, n_estimators=40;, score=-0.061 total time=   0.0s
[CV 2/5] END learning_rate=0.06, max_depth=6, n_estimators=40;, score=-0.068 total time=   0.0s
[CV 3/5] END learning_rate=0.06, max_depth=6, n_estimators=40;, score=-0.059 total time=   0.0s
[CV 4/5] END learning_rate=0.06, max_depth=6, n_estimators=40;, score=-0.054 total time=   0.0s
[CV 5/5] END learning_rate=0.06, max_depth=6, n_estimators=40;, score=-0.0

In [25]:
# Imprimir los resultados
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor: -0.051855 usando {'learning_rate': 0.08, 'max_depth': 8, 'n_estimators': 60}


In [30]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
X_train_xgb_prueba = X_train.copy()
X_train_xgb_prueba = X_train_xgb_prueba.reset_index()
y_train_xgb_prueba = y.copy()
y_train_xgb_prueba = y_train_xgb_prueba.reset_index()


X_train_xgb_prueba = X_train_xgb_prueba.drop('index', axis=1)
y_train_xgb_prueba = y_train_xgb_prueba.drop('index', axis=1)

In [34]:
mse_test_list_total = dict()
mse_train_list_total = dict()
for i in [20,30,40,50,60,70,80,90,100]:
    print(i)
    mse_test_list = []
    mse_train_list = []
    for train_index, test_index in kf.split(X_train_xgb_prueba):
        X_train_cv, X_test_cv = X_train_xgb_prueba.iloc[train_index], X_train_xgb_prueba.iloc[test_index]
        y_train_cv, y_test_cv = y_train_xgb_prueba.iloc[train_index], y_train_xgb_prueba.iloc[test_index]
        
        # Inicializar el modelo de clasificación
        xgboost_cv = xgb.XGBRegressor(learning_rate=0.08, max_depth=8, n_estimators=i)
    
        # Ajustar el modelo con los datos de entrenamiento
        xgboost_cv.fit(X_train_cv, y_train_cv)
    
        # Hacer predicciones en los datos de prueba
        y_pred_test_cv = xgboost_cv.predict(X_test_cv)
        y_pred_train_cv = xgboost_cv.predict(X_train_cv)
    
        # Calcular la precisión y agregarla a la lista de puntuaciones
        mse_test = mean_squared_error(y_test_cv, y_pred_test_cv)
        mse_train = mean_squared_error(y_train_cv, y_pred_train_cv)
        #print(y_test_cv)
        #print(y_pred_cv)
        mse_test_list.append(mse_test)
        mse_train_list.append(mse_train)
        
    mse_cv_test = np.mean(mse_test_list)
    mse_test_list_total[i] = mse_cv_test
    mse_cv_train = np.mean(mse_train_list)
    mse_train_list_total[i] = mse_cv_train
print(mse_cv_test)
print(mse_cv_train)

df_curve = pd.DataFrame(list(mse_test_list_total.items()), columns=['Index', 'ErrorTest'])
df_curve_2 = pd.DataFrame(list(mse_train_list_total.items()), columns=['Index', 'ErrorTrain'])
df_curve = pd.merge(df_curve,df_curve_2,on='Index',how='left')
#df_curve = pd.DataFrame(mse_test_list_total)

20
30
40
50
60
70
80
90
100
0.05147401812195862
0.009456552409827288


In [35]:
df_curve

,Index,ErrorTest,ErrorTrain
0,20,0.061544,0.036664
1,30,0.055110,0.024287
2,40,0.052816,0.018409
3,50,0.052088,0.014831
4,60,0.051855,0.012929
5,70,0.051712,0.011785
6,80,0.051618,0.010851
7,90,0.051501,0.010112
8,100,0.051474,0.009457


In [36]:
# Supongamos que tienes un DataFrame llamado 'df' con columnas "indice", "columna_A" y "columna_B"

# Convertir el DataFrame de pandas a un formato aceptado por Altair
df_altair = pd.melt(df_curve, id_vars=['Index'], value_vars=['ErrorTest', 'ErrorTrain'])

# Crear el gráfico de líneas divididas por color en Altair
line_chart = alt.Chart(df_altair).mark_line(point=True).encode(
    x='Index:Q',
    y='value:Q',
    color='variable:N'
).properties(
    width=600,
    height=400,
    title='Gráfico de Líneas para Columnas A y B'
)

# Mostrar el gráfico
line_chart

alt.Chart(...)

## CTR